In [ ]:
import mysql.connector

connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="12345678"
)

cursor = connection.cursor()
cursor.execute("CREATE DATABASE IF NOT EXISTS imdb;")


# create tables in the DB
create_movies_table = '''create table IF NOT EXISTS movies 
( 
genre varchar(200), 
year int,
title varchar(300), 
duration varchar(10),
certificate varchar(10),
rating float,
voting int
) '''
cursor.execute("use imdb")
cursor.execute(create_movies_table)

connection.commit()
print('Tables successfully created')

In [ ]:
import glob
import os
import pandas as pd
files = glob.glob(os.path.join(r"F:\guvi\Capstone Projects\IMDB-project\data\csv", "*.csv"))
all_dfs = []
for file in files:
    data = pd.read_csv(file)
    all_dfs.append(data)

merged_df = pd.concat(all_dfs, ignore_index=True)
merged_df.drop(columns=["Unnamed: 0"], axis=0,inplace=True)

merged_df.to_csv(r"F:\guvi\Capstone Projects\IMDB-project\data\csv\01_all_merged.csv")

In [ ]:
import numpy as np
import re
str = 'TV-PG' #'1h 40m'
res = re.match(r'(\d+)h\s*(\d*)m?', str)
print(res)
if res:
    hours = int(res.group(1))
    minutes = int(res.group(2)) if res.group(2) else 0
mins = hours * 60 + minutes
print(mins)

In [ ]:
import numpy as np
import re
import pandas as pd

# convert string to minutes
def get_minutes(str):
    if pd.isna(str) or str == "" or str == "None":
        return 0#np.nan
    res = re.match(r'(\d+)h\s*(\d*)m?', str)
    if res:
        hours = int(res.group(1))
        minutes = int(res.group(2)) if res.group(2) else 0
        return hours * 60 + minutes
    return 0 #np.nan


def process_pd(merged_df):
    #Remove leading numbers from "Title"
    merged_df["title"] = merged_df["title"].str.replace(r"^\d+\.\s*", "", regex=True)
    merged_df["duration"] = merged_df["duration"].astype(str).apply(get_minutes).fillna(0)
    return merged_df

merged_df = pd.read_csv(r"F:\guvi\Capstone Projects\IMDB-project\data\csv\01_all_merged.csv")
#merged_df['duration'].unique()
processed = process_pd(merged_df)
processed

,Unnamed: 0,genre,year,title,duration,certificate,rating,voting
0,0,Action,2024,Gladiator II,148,15,6.5,253000
1,1,Action,2024,Dune: Part Two,166,12A,8.5,643000
2,2,Action,2024,Twisters,122,12A,6.5,176000
3,3,Action,2024,Kraven the Hunter,127,15,5.5,63000
4,4,Action,2024,The Ministry of Ungentlemanly Warfare,122,15,6.8,139000
...,...,...,...,...,...,...,...,...
5272,5272,Talk-Show,2024,Paul Molinar: Supercut,129,NaN,0.0,0
5273,5273,Talk-Show,2024,Night of Recovery: Live from the Waynesboro Th...,121,16+,0.0,0
5274,5274,Talk-Show,2024,LIVE Review EP1,69,NaN,0.0,0
5275,5275,Talk-Show,2024,The Poetry of Clark L. Stanton,0,NaN,0.0,0


In [2]:
import pandas as pd
import numpy as np
import re
import mysql.connector
import sqlalchemy
from sqlalchemy import create_engine

# convert string to minutes
def get_minutes(str):
    if pd.isna(str) or str == "" or str == "None":
        return 0#np.nan
    res = re.match(r'(\d+)h\s*(\d*)m?', str)
    if res:
        hours = int(res.group(1))
        minutes = int(res.group(2)) if res.group(2) else 0
        return hours * 60 + minutes
    return 0 #np.nan


def process_pd(merged_df):
    #Remove leading numbers from "Title"
    merged_df["title"] = merged_df["title"].str.replace(r"^\d+\.\s*", "", regex=True)
    merged_df["duration"] = merged_df["duration"].astype(str).apply(get_minutes).fillna(0)
    merged_df.drop(columns=["Unnamed: 0"], axis=0,inplace=True)
    return merged_df

db_user = 'root'
db_password = '12345678'
db_host = 'localhost' 
db_name = 'imdb'

engine_url = f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}"
engine = sqlalchemy.create_engine(engine_url)
print(f"Successfully created SQLAlchemy engine for {db_name} on {db_host}")
print("-" * 30)

try:
    merged_df = pd.read_csv(r"F:\guvi\Capstone Projects\IMDB-project\data\csv\01_all_merged.csv")
    processed_df = process_pd(merged_df)
    print(processed_df)

    processed_df.to_sql(name='movies', con=engine,if_exists='replace',index=False)
    print(f"Dataframe successfully inserted into table 'Movies'")
    
    print("Data inserted successfully.")
except ImportError as e:
    print(f"Error: A required library is not installed. {e}")
    print("Please install it using pip, e.g., 'pip install pymysql' or 'pip install mysql-connector-python'")
except Exception as e:
    print(f"An error occurred: {e}")
    print("Please check your database connection details, table structure, and ensure the MySQL server is running.")




Successfully created SQLAlchemy engine for imdb on localhost
------------------------------
          genre  year                                              title  \
0        Action  2024                                       Gladiator II   
1        Action  2024                                     Dune: Part Two   
2        Action  2024                                           Twisters   
3        Action  2024                                  Kraven the Hunter   
4        Action  2024              The Ministry of Ungentlemanly Warfare   
...         ...   ...                                                ...   
5272  Talk-Show  2024                             Paul Molinar: Supercut   
5273  Talk-Show  2024  Night of Recovery: Live from the Waynesboro Th...   
5274  Talk-Show  2024                                    LIVE Review EP1   
5275  Talk-Show  2024                     The Poetry of Clark L. Stanton   
5276  Talk-Show  2024                            Radyo Agila: Kapitbahay